# Building TensorFlow to Support CPU Instruction Sets

This is what I got when I run TensorFlow in Python using the default build.
```Python
import tensorflow as tf
tf.Session()
```

```
2017-06-09 03:12:49.400281: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2017-06-09 03:12:49.400328: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2017-06-09 03:12:49.400344: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2017-06-09 03:12:49.400356: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2017-06-09 03:12:49.400377: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.
```

Hence, I decided to build TensorFlow to support these instructions. TensorFlow might be faster on my machine if its able to use these instruction sets.  

Note: The example below will be tailored for CUDA(GPU), Ubuntu 16.04 64bit and Python 3. You can skip or change some stuff to tailored it for your setup. Its not that hard, you should be able to figure it out.  

Original tutorial, check out this link for building from sources: https://www.tensorflow.org/install/install_sources

## 1) Installing Dependencies

Just run these commands:  

### A) Installing Bazel  

Check out this link: https://bazel.build/versions/master/docs/install-ubuntu.html

```
sudo apt-get install curl
echo "deb [arch=amd64] http://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
curl https://bazel.build/bazel-release.pub.gpg | sudo apt-key add -
sudo apt-get update
sudo apt-get install bazel
sudo apt-get upgrade bazel
```

### B) Installing Python dependencies

```
sudo apt-get install python3-numpy python3-dev python3-pip python3-wheel
```

### C) Installing CUDA (For GPU)

Visit this [link]() for installing CUDA and setting up your machine for TensorFlow with GPU.

## 2) Configure TensorFlow Build

First, clone the repository, cd into it and run configure.
```
git clone https://github.com/tensorflow/tensorflow
cd tensorflow
./configure
```

Then, you just need to answer all the questions.  
Note: follow the instructions and look up if your machine supports the technology before answering.  
These are my answers for the questions:  

### A) Specify Location of Python
```
You have bazel 0.5.1 installed.
Please specify the location of python. [Default is /usr/bin/python]: /usr/bin/python3
Found possible Python library paths:
  /usr/local/lib/python3.5/dist-packages
  /usr/lib/python3/dist-packages
Please input the desired Python library path to use.  Default is [/usr/local/lib/python3.5/dist-packages]
                                      
Using python library path: /usr/local/lib/python3.5/dist-packages
```

### B) MKL Support
Link for MKL: [Latest at this point of writing](https://software.intel.com/en-us/articles/intel-math-kernel-library-intel-mkl-2017-install-guide)
```
Do you wish to build TensorFlow with MKL support? [y/N] y
MKL support will be enabled for TensorFlow
Do you wish to download MKL LIB from the web? [Y/n] y
```

### C) Bazel Option
Using the default for now, will specify the options later when using bazel build.
```
Please specify optimization flags to use during compilation when bazel option "--config=opt" is specified [Default is -march=native]:
```

### D) Jemalloc
jemalloc is a general purpose malloc(3) implementation that emphasizes fragmentation avoidance and scalable concurrency support.  
Link here: http://jemalloc.net/
```
Do you wish to use jemalloc as the malloc implementation? [Y/n] y
jemalloc enabled
```

### E) Google Cloud Platform, Hadoop
No, don't need them at the moment.
```
Do you wish to build TensorFlow with Google Cloud Platform support? [y/N] n
No Google Cloud Platform support will be enabled for TensorFlow
Do you wish to build TensorFlow with Hadoop File System support? [y/N] n
No Hadoop File System support will be enabled for TensorFlow
```

### F) XLA JIT Compiler
Link: https://www.tensorflow.org/performance/xla/
```
Do you wish to build TensorFlow with the XLA just-in-time compiler (experimental)? [y/N] y
XLA JIT support will be enabled for TensorFlow
```

### G) VERBS-RDMA
Link: https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/verbs
```
Do you wish to build TensorFlow with VERBS support? [y/N] n
No VERBS support will be enabled for TensorFlow
```

### H) OpenCL
Link: https://www.khronos.org/opencl/
```
Do you wish to build TensorFlow with OpenCL support? [y/N] y
OpenCL support will be enabled for TensorFlow
```

### I) CUDA (GPU only, you can skip this)
Select yes if you have installed CUDA. Check out this link for CUDA Installation Instructions: http://waynedev.me/jupyter-setting-up-cuda-for-tensorflow-gpu  
Note for clang: Does not work, can't access BoringSSL for some reason.  

Compute capability for your GPU: https://developer.nvidia.com/cuda-gpus
```
Do you wish to build TensorFlow with CUDA support? [y/N] y
CUDA support will be enabled for TensorFlow
Do you want to use clang as CUDA compiler? [y/N] n
nvcc will be used as CUDA compiler
Please specify the CUDA SDK version you want to use, e.g. 7.0. [Leave empty to default to CUDA 8.0]:   
Please specify the location where CUDA  toolkit is installed. Refer to README.md for more details. [Default is /usr/local/cuda]: 
Please specify which gcc should be used by nvcc as the host compiler. [Default is /usr/bin/gcc]: 
Please specify the cuDNN version you want to use. [Leave empty to default to cuDNN 6.0]: 
Please specify the location where cuDNN  library is installed. Refer to README.md for more details. [Default is /usr/local/cuda]: 
Please specify a list of comma-separated Cuda compute capabilities you want to build with.
You can find the compute capability of your device at: https://developer.nvidia.com/cuda-gpus.
Please note that each additional compute capability significantly increases your build time and binary size.
[Default is: "3.5,5.2"]: 6.1
Please specify which C++ compiler should be used as the host C++ compiler. [Default is /usr/bin/g++]: 
Please specify which C compiler should be used as the host C compiler. [Default is /usr/bin/gcc]: 
Please specify the location where ComputeCpp for SYCL 1.2 is installed. [Default is /usr/local/computecpp]: 
```

### J) MPI Support (Message Passing Interface)
Link: http://mpitutorial.com/tutorials/mpi-introduction/
```
Do you wish to build TensorFlow with MPI support? [y/N] n
MPI support will not be enabled for TensorFlow
Configuration finished
```

## 3) Build using Bazel
This part will take some time, took me approximately 45 minutes.  
Choose the type of build for your machine. The last one is a customized build.

Basic build:
```
bazel build --config=opt tensorflow/tools/pip_package:build_pip_package
```

Build with GPU:
```
bazel build --config=opt --config=cuda //tensorflow/tools/pip_package:build_pip_package 
```

Build with CPU instructions for my machine:
```
bazel build --config=opt --config=cuda --copt=-msse4.2 --copt=-msse4.1 --copt=-mavx --copt=-mavx2 --copt=-mfma --copt=-mfpmath=both tensorflow/tools/pip_package:build_pip_package
```

Note: Add these based on your machine  
Link: https://gcc.gnu.org/onlinedocs/gcc-4.5.3/gcc/i386-and-x86_002d64-Options.html  

--config=cuda --> GPU  
--copt=msse4.1 --> CPU SSE4.1  
--copt=msse4.2 --> CPU SSE4.2  
--copt=mavx --> CPU AVX  
--copt=mavx2 --> CPU AVX2  
--copt=mfma --> CPU FMA  
--copt=mfpmath=both --> Using both SSE and 387

## 4) Install TensorFlow

Run this to build a pip package, then install using pip. You might want to consider using a virtual environment such as virtualenv, docker or conda just in case there's something wrong with the build.  

```
bazel-bin/tensorflow/tools/pip_package/build_pip_package /tmp/tensorflow_pkg
pip install /tmp/tensorflow_pkg/*.whl
```

Note: Using \*.whl because the name differs depending on the build.

## 5) Test the build

Start python and run this piece of code, should output a "DONE" nicely.  
```Python
import tensorflow as tf
done = tf.constant("DONE")
sess = tf.Session()
print(sess.run(done))
```